In [1]:
from pandas_datareader.data import DataReader
from datetime import date # Date & time functionality 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn-dark')
import seaborn as sns


import pypfopt
pypfopt.__version__
from pypfopt import risk_models
from pypfopt import plotting
from pypfopt import expected_returns
from pypfopt import EfficientFrontier
from pypfopt import DiscreteAllocation
from pypfopt import CLA, plotting
from pypfopt.risk_models import CovarianceShrinkage
from pypfopt import objective_functions

In [48]:
import pandas_datareader as web
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices
tickers = ['BRK-B','V','BABA','FB','TCEHY','VOO'] #'BABA','TCEHY','FB'
#'QQQ','RSP','SPY','XLP','XLV','CLOU','ICLN','ARKK', 'BABA','TCEHY','FB'
thelen = len(tickers)

price_data = []
for ticker in range(thelen):
    prices = web.DataReader(tickers[ticker], start='2012-01-01', end = date.today(), data_source='yahoo')
    price_data.append(prices.assign(ticker=ticker)[['Adj Close']])

df_stocks = pd.concat(price_data, axis=1)
df_stocks.columns=tickers
df_stocks.tail()

,BRK-B,V,BABA,FB,TCEHY,VOO
Date,,,,,,
2021-11-11,284.649994,210.419998,167.850006,327.739990,62.459999,426.399994
2021-11-12,285.989990,212.089996,166.809998,340.890015,62.700001,429.570007
2021-11-15,284.670013,212.300003,166.539993,347.559998,62.490002,429.769989
2021-11-16,283.359985,215.179993,168.429993,342.959991,64.620003,431.410004
2021-11-17,282.029999,202.889999,161.244995,342.005005,64.080002,430.339996


##### Full model (Maximising Sharp Ratio)

In [49]:
mu = expected_returns.capm_return(df_stocks)
S = risk_models.CovarianceShrinkage(df_stocks).ledoit_wolf()

ef = EfficientFrontier(mu, S) # Every stock should be more than 3% at least and no more than 50%, if you add ",weight_bounds=(0.03, 0.5)"



"""
While this portfolio seems like it meets our objectives, we might be worried by the fact that a lot of the tickers have been assigned zero weight. In effect, the optimizer is "overfitting" to the data you have provided
-- you are much more likely to get better results by enforcing some level of diversification. One way of doing this is to use L2 regularisation – essentially, adding a penalty on the number of near-zero weights.
"""

# Larger gamma pulls portfolio weights towards an equal allocation.
ef.add_objective(objective_functions.L2_reg, gamma=0.01)
#ef.efficient_risk(0.15)
#ef.efficient_return(target_return=0.12)

#raw_weights = ef.min_volatility() #If you would like to minimise volatility instead
raw_weights = ef.max_sharpe() # Maximises Sharpe Ratio



cleaned_weights = ef.clean_weights()
print(pd.Series(dict(cleaned_weights)).to_frame('Weights').reset_index())

   index  Weights
0  BRK-B  0.14648
1      V  0.14477
2   BABA  0.33985
3     FB  0.17701
4  TCEHY  0.10270
5    VOO  0.08919


/usr/local/lib/python3.9/site-packages/pypfopt/efficient_frontier/efficient_frontier.py:257: UserWarning:

max_sharpe transforms the optimization problem so additional objectives may not work as expected.



In [50]:
ef.portfolio_performance(verbose=True);

Expected annual return: 26.2%
Annual volatility: 19.8%
Sharpe Ratio: 1.22


In [51]:
latest_prices = get_latest_prices(df_stocks)

print('How much money do you want to invest in USD?')
amount = input()

da = DiscreteAllocation(raw_weights, latest_prices, total_portfolio_value= int(amount))

allocation, leftover = da.greedy_portfolio()
print("Discrete allocation:", allocation)
print("Funds remaining: ${:.2f}".format(leftover))

How much money do you want to invest in USD?
7000
Discrete allocation: {'BABA': 14, 'FB': 3, 'BRK-B': 4, 'V': 5, 'TCEHY': 11, 'VOO': 2}
Funds remaining: $8.43


In [52]:
dict_cleaned_weights = dict(cleaned_weights)
portfolio_weights = pd.Series(dict(cleaned_weights)).to_frame('Weights').reset_index().rename(columns = {'index': 'Ticker'})
shares_to_buy = pd.Series(allocation).to_frame(f'USD: {amount}').reset_index().rename(columns = {'index': 'Ticker'})
pf_weights_and_shares = pd.merge(portfolio_weights, shares_to_buy, on="Ticker")
pf_weights_and_shares

,Ticker,Weights,USD: 7000
0,BRK-B,0.14648,4
1,V,0.14477,5
2,BABA,0.33985,14
3,FB,0.17701,3
4,TCEHY,0.10270,11
5,VOO,0.08919,2


In [53]:
display(pf_weights_and_shares)
print(f"Amount I want to invest: ${amount}")
print("Funds remaining after investment: ${:.2f}".format(leftover))
print()
import plotly.express as px
print(ef.portfolio_performance(verbose=True))

fig = px.pie(pf_weights_and_shares, values='Weights', names='Ticker', color_discrete_sequence=px.colors.sequential.RdBu, 
             hover_data=['Ticker'], labels={'Ticker':'Symbol'}, width=800, height=400)
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()


,Ticker,Weights,USD: 7000
0,BRK-B,0.14648,4
1,V,0.14477,5
2,BABA,0.33985,14
3,FB,0.17701,3
4,TCEHY,0.10270,11
5,VOO,0.08919,2


Amount I want to invest: $7000
Funds remaining after investment: $8.43

Expected annual return: 26.2%
Annual volatility: 19.8%
Sharpe Ratio: 1.22
(0.26175974241801475, 0.19816486578366233, 1.2199929662705458)


In [54]:
0.4765 * 4400

2096.6